# Part-of-speech tagging with recurrent neural networks using Keras

## Overview

We will to see how to use the Keras library to build a recurrent neural network (RNN) model that labels part-of-speech (POS) tags for words in sentences. 

### Part-of-speech (POS) tagging

A part-of-speech tag is the syntactic category associated with a particular word in a sentence, such as a noun, verb, preposition, determiner, adjective or adverb. Part-of-speech tagging is a fundamental task in natural language processing; see the [chapter in Juraksky & Martin's *Speech and Language Processing*](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more background. POS tagging is a common pre-processing step in many NLP pipelines. For example, words with certain POS tags are more important than other words for capturing the content of a text (e.g. nouns and verbs carry more semantic meaning than grammatical words like prepositions and determiners), so models often take this into account when predicting the topic, sentiment, or some other categorical dimensions of a text. Start-of-the art models are quite successful, reaching near-perfect accuracy in the tags assigned to words. This notebook will show how to put together a simple tagger that uses a Recurrent Neural Network, though it does not perform as well as more advanced models.

### Recurrent Neural Networks (RNNs)

RNNs are a general framework for modeling sequence data and are particularly useful for natural language processing tasks. At a high level, RNN encode sequences via a set of parameters (weights) that are optimized to predict some output variable. This notebook demonstrates the code needed to assemble an RNN model using the Keras library, as well as some data processing tools that facilitate building the model. 

If you understand how to structure the input and output of the model, and know the fundamental concepts in machine learning, then a high-level understanding of how an RNN works is sufficient for using Keras. You'll see that most of the code here is actually just data manipulation, and I'll visualize each step in this process. The code used to assemble the RNN itself is more minimal. It is of course useful to know the technical details of the RNN, so you can theorize on the results and innovate the model to make it better. For a better understanding of RNNs and neural networks in general, see the resources at the bottom of the notebook.

Here an RNN will be used to encode a sentence and assign a POS tag to each word. The model shown here is applicable to any dataset with a one-to-one mapping between the inputs and outputs. This involves any task where for each sequential unit (here, a word), there is some output unit (here, a POS tag) that should be assigned to that input unit.


## Dataset

The [Brown Corpus](http://www.hit.uib.no/icame/brown/bcm.html) (download through NLTK [here](http://www.nltk.org/nltk_data/)) is a popular NLP resource that consists of 500 texts from a variety of sources, including news reports, academic essays, and fiction. Every word in the texts has been annotated with a POS tag. There are different POS annotation schemes provided in the corpus, which differ in the number of tags assigned. Here I use coarse-grained tags, of which there are eleven unique tags (for example, some schemes might split up the coarse-grained "VERB" tag into fine-grained tags based on the specific tense of the verb). I set up the dataset so that each entry is a single sentence. The code below loads a sample of 100 sentences from the corpus, so see the above [link](http://www.helsinki.fi/varieng/CoRD/corpora/BROWN/) to get the full dataset.

In [ ]:
from __future__ import print_function #Python 2/3 compatibility for print statements
import pandas as pd
pd.set_option('display.max_colwidth', 170) #widen pandas rows display

**Exercise :** Load the dataset with the path dataset/example_train_brown_corpus.csv and save it in a variable train_sents. Load only the first 100 stories to relieve your computer.

In [ ]:
'''Load the dataset'''
### ENTER YOUR CODE HERE (1 line)

### END 
train_sents.head()

### Preprocessing 
We split each space and put the words in lower case for Tokenized_Sentence. 
For Tagged_Sentence, we simply apply a spilt.

In [ ]:
#Get the word tokens and tags into a readable list format
train_sents['Tokenized_Sentence'] = train_sents['Tokenized_Sentence'].apply(lambda sent: sent.lower().split("\t"))
train_sents['Tagged_Sentence'] = train_sents['Tagged_Sentence'].apply(lambda sent: sent.split("\t"))

train_sents[:10]

## Preparing the data

The sentences have already been tokenized into words, so both the words in each sentence and the corresponding tags are represented as lists.

###  Lexicons

We need to assemble lexicons for both the words and tags. The term "lexicon" usually refers specifically to the words in a model, but here I use it generally to mean a mapping between strings and numerical indices, which applies to the POS tags as well (I'll distinguish between the "words lexicon" and "tags lexicon"). Each word/tag is assigned a numerical index that can be read by the model. For the words lexicon, since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. For the tags, we'll include all of them in the model since these are the output classes we are trying to predict. There are only 11 tags in this dataset. An &lt;UNK&gt; tag is included, even though it doesn't actually appear in the dataset; this isn't a problem, because the model will learn not to predict it.

**Exercise :** Create a make_lexicon function. This function return a lexicon for the words in the sentences as well as the tags. Refer you on previous chapter if you don't know how to create a lexicon, it's exactly the same method.

In [ ]:
'''Create a lexicon for the words in the sentences as well as the tags'''

def make_lexicon(token_seqs, min_freq=1):
    ### ENTER YOUR CODE HERE (+- 10lines)
    
    ###END 


Let's make a lexicon for words and another for tags. You should have something like this :  

````
WORDS: {'the': 2, 'fulton': 3, 'county': 4, 'grand': 5, 'jury': 6, 'said': 7, 'friday': 8, 'an': 9, 'investigation': 10, 'of': 11, "atlanta's": 12, 'recent': 13,`... } ````

and for the tags : 
````
TAGS: {'DET': 2, 'NOUN': 3, 'ADJ': 4, 'VERB': 5, 'ADP': 6, '.': 7, 'ADV': 8, 'CONJ': 9, 'PRT': 10, 'PRON': 11, 'NUM': 12, '<UNK>': 1}````


In [ ]:
words_lexicon = make_lexicon(train_sents['Tokenized_Sentence'])
print("WORDS:", words_lexicon)

tags_lexicon = make_lexicon(train_sents['Tagged_Sentence'])
print("TAGS:", tags_lexicon)

**Exercise :** Save your lexicon's ! If you don't remember how to do it, look in the previous lesson. 

In [ ]:
### ENTER YOUR CODE HERE ( +- 5 lines )

### END 

Because the model will output tags as indices, we'll obviously need to map each tag number back to its corresponding string representation in order to later interpret the output. We'll reverse the tags lexicon to create a lookup table to get each tag from its index.

**Exercise :**  Create a get_lexicon_lookup function that returns a dictionary where the string representation of a lexical element can be retrieved from its numerical index. If you don't remember how to do it, look in the previous lesson. 

In [ ]:
'''Make a dictionary where the string representation of a lexicon item can be retrieved from its numerical index'''

def get_lexicon_lookup(lexicon):
    ### ENTER YOUR CODE HERE 

    
    ### END 


Let's check if everything's okay. You should get this 
````
{2: 'DET', 3: 'NOUN', 4: 'ADJ', 5: 'VERB', 6: 'ADP', 7: '.', 8: 'ADV', 9: 'CONJ', 10: 'PRT', 11: 'PRON', 12: 'NUM', 1: '<UNK>'}
````

In [ ]:
tags_lexicon_lookup = get_lexicon_lookup(tags_lexicon)
print(tags_lexicon_lookup)

###  From strings to numbers

We use the lexicons to transform the word and tag sequences into lists of numerical indices.  
**Exercise :** Builds a token_to_idxs() function that transforms word sequences into numbers. If you don't remember how to do it, look at the previous lesson

In [ ]:
 ### ENTER YOUR CODE HERE 

    
### END

Let's test this !

In [ ]:
train_sents['Sentence_Idxs'] = tokens_to_idxs(train_sents['Tokenized_Sentence'], words_lexicon)
train_sents['Tag_Idxs'] = tokens_to_idxs(train_sents['Tagged_Sentence'], tags_lexicon)
train_sents[['Tokenized_Sentence', 'Sentence_Idxs', 'Tagged_Sentence', 'Tag_Idxs']][:10]

If all goes well, you should get this:  
![capture](../img/capt02.png)

###  Numerical lists to matrices

Finally, we need to put the input sequences into matrices for training. There will be separate matrices for the word and tag sequences, where each row is a sentence and each column is a word (or tag) index in that sentence. This matrix format is necessary for the model to process the sentences in batches as opposed to one at a time, which significantly speeds up training. However, each sentence has a different number of words, so we create a padded matrix equal to the length on the longest sentence in the training set. For all sentences with fewer words, we prepend the row with zeros representing an empty word (and tag) position. This is why the number 0 was not assigned as an index in the lexicons. We can specify to Keras to ignore these zeros during training.

**Exercise :** Create a pad_idx_seqs function that will return a sequence. This function will have two parameters, the sequence index and the other one which will be the length of the longest string. 

In [ ]:
from keras.preprocessing.sequence import pad_sequences
###Enter Your code here (+- 3 lines)

###End

**Exercise :** Create a max_seq_len variable that will contain the value of the word number in the largest string. Get length of longest sequence. (59)

In [ ]:
###Enter Your code here (1 line)

### End 
print(max_seq_len)

Let's create the sequences !

In [ ]:
train_padded_words = pad_idx_seqs(train_sents['Sentence_Idxs'], 
                                  max_seq_len + 1) #Add one to max length for offsetting sequence by 1
train_padded_tags = pad_idx_seqs(train_sents['Tag_Idxs'],
                                 max_seq_len + 1)  #Add one to max length for offsetting sequence by 1

print("WORDS:\n", train_padded_words)
print("SHAPE:", train_padded_words.shape, "\n")

print("TAGS:\n", train_padded_tags)
print("SHAPE:", train_padded_tags.shape, "\n")

### Defining the input and output

In this approach, for each word in a sentence, we predict the tag for that word based on two types of input: 

1. all the words in the sentence up to that point, including that current word, and 
2. all the previous tags in the sentence. 

So for a given position in the sentence *idx*, the input is train_padded_words[idx] and train_padded_tags[idx-1], and the output is train_padded_tags[idx]. The example below shows this alignment for the first sentence in the dataset.


In [ ]:
import numpy
pd.DataFrame(list(zip(train_sents['Tokenized_Sentence'].loc[0],
                          ["-"] + train_sents['Tagged_Sentence'].loc[0],
                          train_sents['Tagged_Sentence'].loc[0])),
                 columns=['Input Word', 'Input Tag', 'Output Tag'])

This same alignment is shown below for a sentence in the padded matrices. Because of the offsetting in the alignment, the length of the padded matrices will be reduced by one. 

In [ ]:
print(pd.DataFrame(list(zip(train_padded_words[0,1:], train_padded_tags[0,:-1], train_padded_tags[0, 1:])),
                columns=['Input Words', 'Input Tags', 'Output Tags']))

##  Building the model

### Functional API

To set up the model, we'll use Keras [Functional API](https://keras.io/getting-started/functional-api-guide/), which is one of two ways to assemble models in Keras (the alternative is the [Sequential API](https://keras.io/getting-started/sequential-model-guide/), which is a bit simpler but has more constraints). For the POS tagger model, new tags will be predicted from the combination of two input sequences, the words in the sentence and the corresponding tags in the sentence. The Functional API is specifically useful when a model has multiple inputs and/or outputs. A model consists of a series of layers. As shown in the code below, we initialize instances for each layer. Each layer can be called with another layer as input, e.g. Embedding()(input_layer). A model instance is initialized with the Model() object, which defines the initial input and final output layers for that model. Before the model can be trained, the compile() function must be called with the loss function and optimization algorithm specified (see below).



### Exercise 

Create a model that respects this architecture. Apart from concatenate, you know these layers!
1. Input (words)
1. Input (tags)
2. Embedding (words)
2. Embedding (tags)
3. Concatenat ([words, tags])
4. GRU: The recurrent (GRU) hidden layer 
5. (Time Distributed) Dense: An output layer.

In [ ]:
'''Create the model'''

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, TimeDistributed, Dense
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.recurrent import GRU

def create_model(seq_input_len, n_word_input_nodes, n_tag_input_nodes, n_word_embedding_nodes,
                 n_tag_embedding_nodes, n_hidden_nodes, stateful=False, batch_size=None):
    
    #Layers 1 :  Input (words), Input (tags)
    ### ENTER YOUR CODE HERE (2 lines)
    
    #Layers 2 : word_embeddings = Embedding (words): 
    ### ENTER YOUR CODE HERE (+- 2 lines)
   
    ### END
    #Output shape = (batch_size, seq_input_len, n_word_embedding_nodes)
    
    # Layers 2.b : tag_embeddings = Embedding (tags):
    ### ENTER YOUR CODE HERE (+- 2 lines)
  
    ### END
    #Output shape = (batch_size, seq_input_len, n_tag_embedding_nodes)
    
    #Layer 3 : Concatenate
    merged_embeddings = Concatenate(axis=-1, name='concat_embedding_layer')([word_embeddings, tag_embeddings])
    #Output shape =  (batch_size, seq_input_len, n_word_embedding_nodes + n_tag_embedding_nodes)
    
    #Layer 4 : GRU
    ### ENTER YOUR CODE HERE (+- 2 lines)
    
    
    ### END
    #Output shape = (batch_size, seq_input_len, n_hidden_nodes)
    
    #Layer 5 : TimeDistributed
    ### ENTER YOUR CODE HERE (+- 2 lines)
    
    
    ### END
    # Output shape = (batch_size, seq_input_len, n_tag_input_nodes)
    
    # Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(inputs=[word_input, tag_input], outputs=output_layer)
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer='adam')
    
    return model

In [ ]:
model = create_model(seq_input_len=train_padded_words.shape[-1] - 1, #substract 1 from matrix length because of offset
                     n_word_input_nodes=len(words_lexicon) + 1, #Add one for 0 padding
                     n_tag_input_nodes=len(tags_lexicon) + 1, #Add one for 0 padding
                     n_word_embedding_nodes=300,
                     n_tag_embedding_nodes=100,
                     n_hidden_nodes=500)

### Training

Now we're ready to train the model. We'll call the fit() function to train the model for 10 iterations through the dataset (epochs), using a batch size of 20 sentences. Keras reports to cross-entropy loss after each epoch, which should continue to decrease if the model is learning correctly.

In [ ]:
'''Train the model'''

# output matrix (y) has extra 3rd dimension added because sparse cross-entropy function requires one label per row
model.fit(x=[train_padded_words[:,1:], train_padded_tags[:,:-1]], 
          y=train_padded_tags[:, 1:, None], 
          batch_size=20, epochs=5)
model.save_weights('example_model/model_weights.h5') #Save model

## Tagging new sentences

Now that the model is trained, it can be used to predict tags in new sentences in the test set. As opposed to training where we processed multiple sentences at the same time, it will be more straightforward to demonstrate tagging on a single sentence at a time. In Keras, you can duplicate a model by loading the parameters from a saved model into a new model. Here, this new model will have a batch size of 1. It will also process a sentence one word/tag at a time (seq_input_len=1) and predict the next tag, using the stateful=True parameter to remember its previous predictions within that sentence. The other parameters of this prediction model are exactly the same as the trained model, which is why the weights can be readily transferred. To demonstrate prediction performance, I'll load the weights from a saved model previously trained on the full training set of 51606 sentences (as opposed to 100 sentences in the example dataset used above). I'll apply the model to an example test set of 100 sentences that were not observed during training.


In [ ]:
'''Create predictor model with weights from saved model, with batch_size = 1, seq_input_len = 1 and stateful=True'''

# Load word and tag lexicons from the saved model 
with open('pretrained_model/words_lexicon.pkl', 'rb') as f:
    words_lexicon = pickle.load(f)
    
    
with open('pretrained_model/tags_lexicon.pkl', 'rb') as f:
    tags_lexicon = pickle.load(f)
tags_lexicon_lookup = get_lexicon_lookup(tags_lexicon)

predictor_model = create_model(seq_input_len=1,
                               n_word_input_nodes=len(words_lexicon) + 1,
                               n_tag_input_nodes=len(tags_lexicon) + 1,
                               n_word_embedding_nodes=300,
                               n_tag_embedding_nodes=100,
                               n_hidden_nodes=500,
                               stateful=True,
                               batch_size=1)

#Transfer the weights from the trained model
predictor_model.load_weights('pretrained_model/model_weights.h5')

**Exercise :** Load the test set and apply same processing steps performed above for training set. Load dataset in ``dataset/example_test_brown_corpus.csv``

In [ ]:
'''Load the test set and apply same processing steps performed above for training set'''
### ENTER YOUR CODE HERE (5 lines)
test_sents =...
test_sents['Tokenized_Sentence'] =...
test_sents['Tagged_Sentence'] = ...
test_sents['Sentence_Idxs'] = ...
test_sents['Tag_Idxs'] = ...
### END

We'll iterate through the sentences in the test set and tag each of them. For each sentence, we start with an empty list for the predicted tags. For the first word in the sentence, there is no previous tag, so the model reads that word and the empty tag 0 (the padding value). The predict() function returns a probability distribution over the tags, and we pick the tag with the highest probability as the one to assign that word. This tag is appended to our list of predicted tags, and we continue to the next word in the sentence. Because the model is stateful, we can simply provide the current word and most recent tag as input to the predict() function, since its hidden layer has memorized the sequence of words/tags observed so far. After the entire sentence has been tagged, we call reset_states() to clear the values for this sentence so we can process a new sentence. The tag indices are mapped back to their string forms, which I show in the sample below, alongside the correct (gold) tags for comparison.


In [ ]:
'''Predict tags for sentences in test set'''

import numpy

pred_tags = []
for _, sent in test_sents.iterrows():
    tok_sent = sent['Tokenized_Sentence']
    sent_idxs = sent['Sentence_Idxs']
    sent_gold_tags = sent['Tagged_Sentence']
    sent_pred_tags = []
    prev_tag = 0  #initialize predicted tag sequence with padding
    for cur_word in sent_idxs:
        # cur_word and prev_tag are just integers, but the model expects an input array
        # with the shape (batch_size, seq_input_len), so prepend two dimensions to these values
        p_next_tag = predictor_model.predict(x=[numpy.array(cur_word)[None, None],
                                                numpy.array(prev_tag)[None, None]])[0]
        prev_tag = numpy.argmax(p_next_tag, axis=-1)[0]
        sent_pred_tags.append(prev_tag)
    predictor_model.reset_states()

    #Map tags back to string labels
    sent_pred_tags = [tags_lexicon_lookup[tag] for tag in sent_pred_tags]
    pred_tags.append(sent_pred_tags) #filter padding 

test_sents['Predicted_Tagged_Sentence'] = pred_tags

#print sample
for _, sent in test_sents[:10].iterrows():
    print("SENTENCE:\t{}".format("\t".join(sent['Tokenized_Sentence'])), "\n\n")
    print("PREDICTED:\t{}".format("\t".join(sent['Predicted_Tagged_Sentence'])), "\n\n")
    print("GOLD:\t\t{}".format("\t".join(sent['Tagged_Sentence'])), "\n\n")

    

### Evaluation

We can evaluate our model with some of the standard metrics for classification: *precision*, *recall*, and *F1 score*. In the context of this task, precision is the proportion of the predicted tags for a particular class that were correct predictions (i.e. of all the words that were assigned a NOUN tag by the tagger, what percentage of these were actually nouns according to the test set?). Recall is the proportion of correct tags for a particular class that the tagger also predicted correctly (i.e. of all the words in the test set that should have been assigned a NOUN tag, what percentage of these were actually tagged as a NOUN?). F1 score is a weighted average of precision and recall. The scikit-learn package has several of these [evaluation metrics](http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics) available.

In [ ]:
'''Evalute the model by precision, recall, and F1'''

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

if __name__ == '__main__':
    all_gold_tags = [tag for sent_tags in test_sents['Tagged_Sentence'] for tag in sent_tags]
    all_pred_tags = [tag for sent_tags in test_sents['Predicted_Tagged_Sentence'] for tag in sent_tags]
    accuracy = accuracy_score(y_true=all_gold_tags, y_pred=all_pred_tags)
    precision = precision_score(y_true=all_gold_tags, y_pred=all_pred_tags, average='weighted')
    recall = recall_score(y_true=all_gold_tags, y_pred=all_pred_tags, average='weighted')
    f1 = f1_score(y_true=all_gold_tags, y_pred=all_pred_tags, average='weighted')

    print("ACCURACY: {:.3f}".format(accuracy))
    print("PRECISION: {:.3f}".format(precision))
    print("RECALL: {:.3f}".format(recall))
    print("F1: {:.3f}".format(f1))


### Visualizing data inside the model

To help visualize the data representation inside the model, we can look at the output of each layer individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://www.civisanalytics.com/blog/interpreting-visualizing-neural-networks-text-processing/)), but seeing them verifies the model is just a series of transformations from one matrix to another. The get_layer() function lets you retrieve any layer by the name that was assigned to it when creating the model. Below is an example of the output for the tag embedding layer for the first word in the first sentence of the test set. You can do this same thing to view any layer.

In [ ]:
'''Show the output of the tag embedding layer for the first word in the first sentence'''

tag_embedding_layer = Model(inputs=[predictor_model.get_layer('word_input_layer').input,
                                    predictor_model.get_layer('tag_input_layer').input], 
                            outputs=predictor_model.get_layer('tag_embedding_layer').output)
#Show tag embedding used to predict first tag in sequence (word input is first word, tag input is 0)
tag_embedding_output = tag_embedding_layer.predict([numpy.array(test_sents['Sentence_Idxs'][0][0])[None,None], 
                                                    numpy.array(0)[None,None]])
print("TAG EMBEDDINGS OUTPUT SHAPE:", tag_embedding_output.shape)
print(tag_embedding_output[0])

It is also easy to look at the weight matrices that connect the layers. The get_weights() function will show the incoming weights for a particular layer.

In [ ]:
'''Show weights that connect hidden layer to output layer'''

hidden_to_output_weights = predictor_model.get_layer('output_layer').get_weights()[0]
print("HIDDEN-TO_OUTPUT WEIGHTS SHAPE:", hidden_to_output_weights.shape)
print(hidden_to_output_weights)

## Conclusion

Even though this model can accuractely predict many POS tags, state-of-the-art taggers use more sophisticated techniques. For example, where here we predicted a tag just based on the preceding words and tags, [bidirectional layers](https://keras.io/layers/wrappers/#bidirectional) also model the sequence that appears after the given word to additionally inform the prediction. POS tagging can be seen as a shallow version of syntactic parsing, which is a more difficult NLP problem. Where POS tagging outputs a flat sequence with a one-to-one mapping between words and tags, syntatic parsing produces a hierarchical structure where categories consist of multiple-word phrases and phrase categories are embedded inside other phrases. Check out the [chapter from Jurafsky & Martin's book](https://web.stanford.edu/~jurafsky/slp3/14.pdf) if you're interested in learning more about these deeper models of linguistic structure.